In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Input,Flatten, Embedding, Bidirectional, LSTM, Dense, Concatenate, TimeDistributed

In [ ]:
# Parameters
max_sentence_length = 100  # Maximum length of sentence embeddings
embedding_dim = 512        # Dimension of sentence embeddings
max_char_length = 10       # Maximum length of a word in characters
char_vocab_size = 28       # Number of unique characters
num_diacritics = 4         # Number of possible diacritics for each character, including no diacritic

# BiLSTM model
bi_lstm_model = Sequential()
bi_lstm_model.add(Bidirectional(LSTM(64, return_sequences=False), input_shape=(max_sentence_length, embedding_dim)))


# RNN model
rnn_model = Sequential()
rnn_model.add(Input(shape=(408,)))  # Adjust the input shape based on the BiLSTM output and character input
rnn_model.add(Dense(128, activation='relu'))
rnn_model.add(Dense(64, activation='relu'))
rnn_model.add(Dense(max_char_length * num_diacritics, activation='softmax'))
rnn_model.add(tf.keras.layers.Reshape((max_char_length, num_diacritics)))


# Sentence input
sentence_input = Input(shape=(max_sentence_length, embedding_dim))

# Character input
char_input = Input(shape=(max_char_length, char_vocab_size))

# Get BiLSTM output
bi_lstm_output = bi_lstm_model(sentence_input)

# Flatten the character input
flat_char_input = Flatten()(char_input)

# Concatenate BiLSTM output and flattened character input
combined_input = Concatenate()([bi_lstm_output, flat_char_input])

# Get the final diacritics predictions
diacritics_output = rnn_model(combined_input)

# Final model
final_model = Model(inputs=[sentence_input, char_input], outputs=diacritics_output)

# Compile the model
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Model summary
final_model.summary()
